# Create footprint overview of flight campaign

* Merging footprints and nav data

In [1]:
import geopandas as gpd
import pytz
import fiona
import pandas as pd
import glob2, glob
import os

### Setup params

In [2]:
DATADIR = r'N:\response\Restricted_Airborne\MACs\Alaska\ThawTrend-Air_2019\raw_data'
CRS = {'init' :'epsg:4326'}

## Internal settings and functions

In [3]:
# Workaround to read KML files in geopandas
fiona.supported_drivers['KML'] = 'rw'
# Timezone Definitions
utc = pytz.utc
akdt = pytz.timezone('US/Alaska')
pd.set_option('display.max_colwidth', -1)

In [4]:
def load_kml(file, sensor_name='RGB', layer=0, looking='left'):
    df = gpd.read_file(file, driver='KML', layer=layer)
    df['Sensor'] = sensor_name
    df['Looking'] = looking
    df = df.set_index('Name', drop=True)
    return df

#### Read subdirectories

In [14]:
subdirs = glob2.glob(os.path.join(DATADIR, '2019*'))
df_subdirs = pd.DataFrame(subdirs)
display(pd.DataFrame(subdirs))

,0
0,N:\response\Restricted_Airborne\MACs\Alaska\ThawTrend-Air_2019\raw_data\20190714-023051_1_Drew_Point_Flight_plan_v2
1,N:\response\Restricted_Airborne\MACs\Alaska\ThawTrend-Air_2019\raw_data\20190714-031731_1_Drew_Point_Flight_plan_v2
2,N:\response\Restricted_Airborne\MACs\Alaska\ThawTrend-Air_2019\raw_data\20190719-223814_1_Drew_Point_Flight_plan_v3
3,N:\response\Restricted_Airborne\MACs\Alaska\ThawTrend-Air_2019\raw_data\20190719-225022_1_Drew_Point_Flight_plan_v3
4,N:\response\Restricted_Airborne\MACs\Alaska\ThawTrend-Air_2019\raw_data\20190719-230135_16_Methane_Seeps_flight_plan_v2
5,N:\response\Restricted_Airborne\MACs\Alaska\ThawTrend-Air_2019\raw_data\20190719-231506_1_Drew_Point_Flight_plan_v3
6,N:\response\Restricted_Airborne\MACs\Alaska\ThawTrend-Air_2019\raw_data\20190719-231538_16_Methane_Seeps_flight_plan_v2
7,N:\response\Restricted_Airborne\MACs\Alaska\ThawTrend-Air_2019\raw_data\20190720-010202_[ - ]
8,N:\response\Restricted_Airborne\MACs\Alaska\ThawTrend-Air_2019\raw_data\20190720-011048_[ - ]
9,N:\response\Restricted_Airborne\MACs\Alaska\ThawTrend-Air_2019\raw_data\20190720-015948_[ - ]


## Part 2 

#### Select index of directory from table above

In [61]:
idx = 49
s = df_subdirs.loc[idx, 0]
dataset_id = os.path.split(s)[-1]
print(s)

N:\response\Restricted_Airborne\MACs\Alaska\ThawTrend-Air_2019\raw_data\20190731-033159_[Record All]


#### Get filenames of relevant files
* RGB and NIR fooprint kmls
* nav txt file with additional metadata

In [62]:
file_rgb = glob.glob(os.path.join(glob.escape(s), '*_RGB.kml'))[0]
file_nir = glob.glob(os.path.join(glob.escape(s), '*_NIR.kml'))[0]
file_nav = glob.glob(os.path.join(glob.escape(s), '*_nav.txt'))[0]

#### Setup nav file

In [63]:
df_nav = pd.read_csv(file_nav, delimiter='\t')
df_nav.columns = [c.strip() for c in df_nav.columns] # remove trailing whitespaces in Feature name
df_nav['File'] = [f.strip() for f in df_nav['File']] # remove trailing whitespaces in File names
df_nav['Basename'] = [os.path.basename(f) for f in df_nav['File']] # remove trailing whitespaces in File names
df_nav['Time_UTC'] = pd.to_datetime(df_nav['Date'] + ' ' + df_nav['Time'], utc=True)
df_nav['Time_Loc'] = df_nav.Time_UTC.dt.tz_convert(akdt)
df_nav['Time_UTC'] = df_nav['Time_UTC'].apply(lambda x: x.strftime('%Y%m%dT%H:%M%SZ'))
df_nav['Time_Loc'] = df_nav['Time_Loc'].apply(lambda x: x.strftime('%Y%m%dT%H:%M%SZ'))
df_nav = df_nav.set_index('Basename', drop=True)

#### setup kml files

In [64]:
df_concat = pd.concat([load_kml(file_rgb, sensor_name='RGB', layer=0, looking='left'),
                       load_kml(file_rgb, sensor_name='RGB', layer=1, looking='right'),
                       load_kml(file_nir, sensor_name='NIR', layer=0, looking='center')])

#### Join footprints and metadata

In [65]:
df_join = df_nav.join(df_concat).drop(columns=['File', 'Description', 'Easting[m]', 'Northing[m]', 'Zone', 'Date', 'Time']).reset_index()
df_join = gpd.GeoDataFrame(df_join, crs=CRS)

#### Save output

In [66]:
outfile = os.path.join(s, '{f}_footprints_full.shp'.format(f=dataset_id))
df_join.to_file(outfile, driver="ESRI Shapefile")

#### Create and export dissolved dataset

In [67]:
df_dissolved = gpd.GeoDataFrame(geometry=[df_join.unary_union], crs=CRS)
df_dissolved['Dataset'] = dataset_id

In [68]:
outfile_dissolved = os.path.join(s, '{f}_footprints_dissolved.shp'.format(f=dataset_id))
df_dissolved.to_file(outfile_dissolved, driver="ESRI Shapefile")